# Introduction
In this file, spot light search MVPA is applied on Z-stat files from first level analysis I have done on naive space, ART applied images, with each stimuli giving one Z-stat image.

MPVA is applied for all voxels of the brain. For each participant, data is split into the number of runs, with data from one run is used for test, and the rest is used for training. 

## Future works
Testing on various types of classifers, wokring on classifier performance are works to be done.


    

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
import itertools
import multiprocessing
from multiprocessing import Pool
from importlib import reload #import function "reload"
import sys

import warnings
warnings.filterwarnings("ignore")
import os
import glob
from glob import glob
from scipy.io import loadmat
import math
import timeit
from timeit import default_timer as timer

plt.style.use('classic')
%matplotlib inline

In [2]:
from MVPA_func import SpherePoints, SphereValue, LoadAllNii, MVPA_FeatureExtraction, MVPA_Classification, LoopiFeatureDict, multi_run_wrapperLoopiFeatureDict

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

### Load subject list file

In [4]:
subj_list_file='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/subj_simultaneous.txt'
subj_list=list()
f = open(subj_list_file, 'r')
line=f.readline()
while line:
    subj_list.append(line[0:-1])
    oldline=line
    line = f.readline()
subj_list.pop()
subj_list.append(oldline)
f.close()
Nsubj=len(subj_list)

### Load SPM.mat to extract experiment info
This cell loads spm.mat file for each participant and from the data, extracts the number of runs, the path for each stat file, etc

In [5]:
# note: I don't know why the last item is repeated! but it doesn't make a misatke

conditions=['Neu','Neg','Pos','Targ']
ZstatInfo=pd.DataFrame(columns=['Subj','Run','Folder']+conditions)
for subj in subj_list:
    #print('Participant: '+subj)
    # Zstat files and their condisitons are in SPM filesWe repeat loading in the dataframe to avoid dividing the onsets by 2 multiple times ...
    SubjSPMFolder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/'+subj+'/3cond_dist2/'
    os.path.isdir(SubjSPMFolder)
    myZstat=[]
    
    nRuns = len(os.listdir(SubjSPMFolder))
    iRuns = 0
    for myRun in os.listdir(SubjSPMFolder):
        ZstatInfotmp=pd.DataFrame(columns=ZstatInfo.columns,index=[0])
        ZstatInfotmp['Subj'][0]=subj
        
        RunSubjSPM=SubjSPMFolder+myRun+'/SPM.mat'
        ZstatInfotmp['Folder'][0]=SubjSPMFolder+myRun+'/'

        #print('   loading SPM.mat for '+myRun+'...')
        mat = loadmat(RunSubjSPM)
        mdata=mat['SPM'][0,0]
        ndata=mdata['Sess'][0,0]
        odata=ndata['U']
        pdata=odata['name'][0]
        ZstatNames=[pdata[i][0,0][0] for i in range(len(pdata))]
        myZstat.append(ZstatNames)

        ZstatInfotmp['Run'][0]=iRuns+1
        old_ind=len(myZstat[0])+1
        for iCond in reversed(conditions):    
            Start_ind=myZstat[0].index(iCond+'_1')+1
            #print(Start_ind)
            ZstatInfotmp[iCond][0]=['spmT_'+str(val).zfill(4)+'.nii' for val in list(range(Start_ind, old_ind))]#list(range(Start_ind, old_ind))
            old_ind=Start_ind
        ZstatInfo=ZstatInfo.append(ZstatInfotmp,ignore_index=True)
        iRuns = iRuns+1
 

### find Voxel dimension by openning one of the image files


In [6]:
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==1)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
#print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)


(64, 64, 28)


In [7]:
# Calculate the spotlights voxels for all points and keep them in AllPoints 
start = timer()

radius = 1 #spheres have radius 2
tmpPoints = SpherePoints(np.array([26,26,16]) , radius, voxel_dims)
AllZeroSphere = np.zeros((1,tmpPoints.shape[2]))
AllZeroSpherePoints=np.zeros(tmpPoints.shape)
cnt=0
for MyCenter in itertools.product(range(0,voxel_dims[0]), range(0,voxel_dims[1]), range(0,voxel_dims[2])):
    #print(MyCenter)
    myPoints = SpherePoints(np.array(MyCenter) , radius, voxel_dims)
    if myPoints.shape==(3,1,0): myPoints=AllZeroSpherePoints
    if cnt==0:
        AllPoints = np.array(myPoints,ndmin=4)
        cnt=1
    else:
        AllPoints = np.concatenate((AllPoints,np.array(myPoints,ndmin=4)),axis=0)
#     if myPoints.shape!=(3,1,0): sphereVal = data[myPoints[0],myPoints[1],myPoints[2]]
#     else: sphereVal = AllZeroSphere
        
#    if myPoints.shape!=(0,): sphereVal = SphereValue(myPoints,data)
#     if sphereVal.shape!=(0,): sphereValOut=


end = timer()
print(end-start)

305.241091399992


In [44]:
# This code does classification based on spot light search, serial processing, slow
start = timer()
CLconditions=['Neu','Neg','Targ']
radius = 1 #spheres radius


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list_one)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list_one):
    
    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    
    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    for iFeature in range(AllPoints.shape[0]): #iFeature=40836, range(56500,56610):
        SpotLight_performance[iFeature]= LoopiFeatureDict(iFeature,DataArgDict)

end = timer()
print(end-start)
SpotLight_performance[56500:56610]

1630.47824
[0.5269097222222222, 0.5873263888888889, 0.5022569444444444, 0, 0, 0.4072916666666667, 0.5145833333333333, 0.506076388888889, 0.63125, 0.5677083333333333]


In [13]:
# This code does classification based on spot light search, parallel processing, fast
start = timer()
CLconditions=['Neu','Neg','Targ']
radius = 1 #spheres radius


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list_one)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list_one):
    
    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    
    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
    aa1= range(AllPoints.shape[0])
    aa2=[DataArgDict]*len(aa1)
    aa=[*zip(aa1, aa2)]

    if __name__ == "__main__":

        pool = Pool(multiprocessing.cpu_count())
        resultsN = pool.map(multi_run_wrapperLoopiFeatureDict,aa)

end = timer()
print(end-start)
print(resultsN[56500:56510])

229.17325809999602
[0.4682291666666667, 0.562326388888889, 0.4998263888888889, 0, 0, 0.4109375, 0.5413194444444444, 0.49756944444444445, 0.6347222222222222, 0.6092013888888889]
